In [1]:
import ast
import torch
import seaborn as sns
import numpy as np
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt

from torchmetrics.functional.classification import (
    f1_score,
    multilabel_precision,
    multilabel_recall,
    multilabel_average_precision,
    confusion_matrix,
)

from sklearn.metrics import accuracy_score

**Load annotations and results**

In [2]:
# Relevant paths
train_results_file = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/data/local_for_maksim/results/model=slow_r50-w-negatives_e=100_split=train.pkl"
val_results_file = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/data/local_for_maksim/results/model=slow_r50-w-negatives_e=100_split=val.pkl"
metadata_file = "/home/dl18206/Desktop/phd/code/personal/facebook/slowfast/data/local_for_maksim/metadata/metadata.csv"
behavioural_labels_file = "../data/behaviours.txt"
segements_file = "../data/segements.txt"

In [3]:
with open(
    train_results_file,
    "rb",
) as f:
    train_data = pkl.load(f)

with open(val_results_file, "rb") as f:
    val_data = pkl.load(f)

metadata_df = pd.read_csv(metadata_file)

with open(behavioural_labels_file, "rb") as f:
    behaviours = [beh.decode("utf-8").strip() for beh in f.readlines()]

with open(segements_file, "rb") as f:
    segments = [seg.decode("utf-8").strip() for seg in f.readlines()]

In [29]:
def results2df(train_data, val_data, metadata_df):

    # Process subclips
    subclips = []
    for i, split in enumerate([train_data, val_data]):
        for name, pred, feat, label in zip(
            split["names"], split["preds"], split["feats"], split["labels"]
        ):
            subclips.append(
                {
                    "name": name,
                    "split": i,
                    "pred": pred,
                    "feat": feat,
                    "negative": True if sum(label) == 0 else False,
                }
            )

    df = pd.DataFrame(subclips, columns=["name", "split", "pred", "feat", "negative"])

    df["split"] = df.split.map({0: "train", 1: "val"})
    df = df.merge(metadata_df, how="left", left_on="name", right_on="subject_id")

    # Apply sigmoid to predictions
    df["pred"] = df.pred.apply(lambda x: torch.sigmoid(torch.tensor(x)))

    # Convert label from str to int
    df.label = df.label.apply(lambda x: np.array(ast.literal_eval(x)))

    # Add negative
    df["negative"] = df.label.apply(lambda x: sum(x) == 0)

    # Add global location count to dataframe
    df["location_count"] = df.utm.map(df.utm.value_counts())

    # Return train and val dataframes
    train_df = df[df.split == "train"]
    val_df = df[df.split == "val"]

    return train_df, val_df


def print_per_segement_performance(map, segment, show_per_class=False):
    res = []
    for i, (b, s) in enumerate(zip(map, segments)):
        if s == segment:
            res.append({behaviours[i]: b})
    agg_values = []
    for r in res:
        for _, value in r.items():
            agg_values.append(value)
    if show_per_class:
        print(f"{segment}: {np.mean(agg_values):.2f} {res}")
    else:
        print(f"{segment}: {np.mean(agg_values):.2f}")

In [30]:
# This will return dfs with preds, feats, labels, and all metadata
train_df, val_df = results2df(train_data, val_data, metadata_df)

**Metric calculation**

In [41]:
# Train performance
train_map = multilabel_average_precision(
    torch.tensor(np.stack(train_df["pred"])),
    torch.tensor(np.stack(train_df["label"])),
    num_labels=14,
    average="none",
)

print("Train mAP:", round(train_map.mean().item(), 2))
print_per_segement_performance(train_map, "head")
print_per_segement_performance(train_map, "tail")
print_per_segement_performance(train_map, "few_shot")


# Val performance
val_map = multilabel_average_precision(
    torch.tensor(np.stack(val_df["pred"])),
    torch.tensor(np.stack(val_df["label"])),
    num_labels=14,
    average="none",
)
print("\n")
print("Val mAP:", round(val_map.mean().item(), 2))
print_per_segement_performance(val_map, "head")
print_per_segement_performance(val_map, "tail")
print_per_segement_performance(val_map, "few_shot")

Train mAP: 0.99
head: 1.00
tail: 0.99
few_shot: 0.98


Val mAP: 0.44
head: 0.69
tail: 0.46
few_shot: 0.27
